This is Daniel Sherman NLP Project Part 3

I will be using Deep Learning to perform binary classification on the huggingface IMDB dataset

In [15]:
# import all of the python modules/packages you'll need here
import pandas as pd
from datasets import load_dataset
import datasets #huggingface datasets
import nltk #natural language tool kit

#myDataSet = load_dataset("imdb")

#load datasets split into training and testing data
myTrainData_ds = load_dataset("imdb",split="train")
myTestData_ds = load_dataset("imdb", split="test")
#convert datasets to pandas dataframes
myTrainData_df = pd.DataFrame(myTrainData_ds)
myTestData_df = pd.DataFrame(myTestData_ds)
myTrainData_df.head()

,text,label
0,I rented I AM CURIOUS-YELLOW from my video sto...,0
1,"""I Am Curious: Yellow"" is a risible and preten...",0
2,If only to avoid making this type of film in t...,0
3,This film was probably inspired by Godard's Ma...,0
4,"Oh, brother...after hearing about this ridicul...",0


In [16]:
myTestData_df.head()

,text,label
0,I love sci-fi and am willing to put up with a ...,0
1,"Worth the entertainment value of a rental, esp...",0
2,its a totally average film with a few semi-alr...,0
3,STAR RATING: ***** Saturday Night **** Friday ...,0
4,"First off let me say, If you haven't enjoyed a...",0


In [17]:
#introduce stopwords and common characters to increase value of tokens when tokenizing
from nltk.corpus import stopwords
stops = set(stopwords.words('english'))
chars2remove = set([',','!','/','?','"','#',':','.','\'','<','>','(',')'])


myTrainData_df['tokens'] = myTrainData_df['text'].apply(lambda x: nltk.word_tokenize(x))
myTestData_df['tokens'] = myTestData_df['text'].apply(lambda x: nltk.word_tokenize(x))
#myTrainData_df.head()
print('in progress')
#refine tokenization to remove stopwords and common characters
myTrainData_df['tokens'] = myTrainData_df['tokens'].apply(lambda x: [w for w in x if w not in stops])
myTrainData_df['tokens'] = myTrainData_df['tokens'].apply(lambda x: [w for w in x if w not in chars2remove])
myTestData_df['tokens'] = myTestData_df['tokens'].apply(lambda x: [w for w in x if w not in stops])
myTestData_df['tokens'] = myTestData_df['tokens'].apply(lambda x: [w for w in x if w not in chars2remove])
myTrainData_df.head()
print("done")

in progress
done


In [18]:
# https://www.nltk.org/install.html
# https://www.nltk.org/data.html
# import nltk
# nltk.download('wordnet')
# nltk.download('omw-1.4')
# do in Kaggle console to make it work online

# lemmatize to increase standardization of tokenized data for better model training
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
myTrainData_df['tokens'] = myTrainData_df['tokens'].apply(lambda x: [lemmatizer.lemmatize(w, pos="v") for w in x])
myTestData_df['tokens'] = myTestData_df['tokens'].apply(lambda x: [lemmatizer.lemmatize(w, pos="v") for w in x])

myTrainData_df.head()

,text,label,tokens
0,I rented I AM CURIOUS-YELLOW from my video sto...,0,"[I, rent, I, AM, CURIOUS-YELLOW, video, store,..."
1,"""I Am Curious: Yellow"" is a risible and preten...",0,"[``, I, Am, Curious, Yellow, '', risible, pret..."
2,If only to avoid making this type of film in t...,0,"[If, avoid, make, type, film, future, This, fi..."
3,This film was probably inspired by Godard's Ma...,0,"[This, film, probably, inspire, Godard, 's, Ma..."
4,"Oh, brother...after hearing about this ridicul...",0,"[Oh, brother, ..., hear, ridiculous, film, ump..."


In [19]:
myTrainData_df['textclean'] = myTrainData_df['tokens'].apply(lambda x: ' '.join(x))
myTestData_df['textclean'] = myTestData_df['tokens'].apply(lambda x: ' '.join(x))
myTrainData_df.head()

,text,label,tokens,textclean
0,I rented I AM CURIOUS-YELLOW from my video sto...,0,"[I, rent, I, AM, CURIOUS-YELLOW, video, store,...",I rent I AM CURIOUS-YELLOW video store controv...
1,"""I Am Curious: Yellow"" is a risible and preten...",0,"[``, I, Am, Curious, Yellow, '', risible, pret...",`` I Am Curious Yellow '' risible pretentious ...
2,If only to avoid making this type of film in t...,0,"[If, avoid, make, type, film, future, This, fi...",If avoid make type film future This film inter...
3,This film was probably inspired by Godard's Ma...,0,"[This, film, probably, inspire, Godard, 's, Ma...",This film probably inspire Godard 's Masculin ...
4,"Oh, brother...after hearing about this ridicul...",0,"[Oh, brother, ..., hear, ridiculous, film, ump...",Oh brother ... hear ridiculous film umpteen ye...


I will be using GPT2 to classify the reviews with code modified from:

https://colab.research.google.com/github/gmihaila/ml_things/blob/master/notebooks/pytorch/gpt2_finetune_classification.ipynb#scrollTo=6oaNu3HJtTTo

and

https://www.kaggle.com/code/andres6garzon/getting-started-nlp-classification-using-gpt-2

In [20]:

##Code copied and modified from colab notebook above
import io
import os
import torch
from tqdm.notebook import tqdm
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import classification_report, accuracy_score
from transformers import (set_seed,
                          TrainingArguments,
                          Trainer,
                          GPT2Config,
                          GPT2Tokenizer,
                          AdamW, 
                          get_linear_schedule_with_warmup,
                          GPT2ForSequenceClassification)

# Set seed for reproducibility.
set_seed(123)

# Number of training epochs (authors on fine-tuning Bert recommend between 2 and 4).
epochs = 4

# Number of batches - depending on the max sequence length and GPU memory.
# For 512 sequence length batch of 10 works without cuda memory issues.
# For small sequence length can try batch of 32 or higher.
batch_size = 32

# Pad or truncate text sequences to a specific length
# if `None` it will use maximum sequence of word piece tokens allowed by model.
max_length = 60

# Look for gpu to use. Will use `cpu` by default if no gpu found.
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Name of transformers model - will use already pretrained model.
# Path of transformer model - will load your own model from local disk.
model_name_or_path = 'gpt2'

# Dictionary of labels and their id - this will be used to convert.
# String labels to number ids.
labels_ids = {'neg': 0, 'pos': 1}

# How many labels are we using in training.
# This is used to decide size of classification head.
n_labels = len(labels_ids)

print("done")

done


In [21]:
###code from Kaggle notebook above
class DatasetCreator(Dataset):
    def __init__(self, processed_data, train):
        self.data = processed_data
        self.train = train
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        line = self.data.iloc[index]
        if self.train:
            return {'text': line['textclean'], 'label': line['label']}
        else:
            return {'text': line['textclean'], 'label': 0}

print("done")

done


In [22]:
###code from Kaggle notebook above, initializes functions used in GPT2 model
class GPT2_collator(object):
    def __init__(self, tokenizer, max_seq_len=None):
        self.tokenizer = tokenizer
        self.max_seq_len = max_seq_len
        return
    
    def __call__(self, sequences):
        texts = [sequence['text'] for sequence in sequences]
        labels = [int(sequence['label']) for sequence in sequences]
        inputs = self.tokenizer(text=texts,
                                return_tensors='pt',
                                padding=True,
                                truncation=True,
                                max_length=self.max_seq_len)
        inputs.update({'labels': torch.tensor(labels)})       
        return inputs
    
# Function to remove hashtags....etc. Intended to improve performance   
def pre_process(dataset):
    # Remove hashtag
    dataset['textclean'] = dataset['text']#.apply(lambda x: x.replace('#', ' ')) 
    # Remove websites
    #dataset['clean_tweet'] = dataset['clean_tweet'].apply(lambda x: re.sub(r"http\S+", " ", x))
    # Remove emails
    #dataset['clean_tweet'] = dataset['clean_tweet'].apply(lambda x: re.sub(r"\S*@\S*\s?", " ", x))
    # Remove punctuation
    #dataset['clean_tweet'] = dataset['clean_tweet'].apply(lambda x: re.sub(r"[^a-zA-Z]", " ", x))
    # Remove 'RT'
    #dataset['clean_tweet'] = dataset['clean_tweet'].apply(lambda x: re.sub(r"RT\s+", " ", x))
    return dataset

# Function for training
def train(dataloader, optimizer, scheduler, device):
    global model
    model.train()
    predictions_labels = []
    true_labels = []
    total_loss = 0
    
    for batch in tqdm(dataloader, total=len(dataloader)):
        true_labels += batch['labels'].numpy().flatten().tolist()
        batch = {k:v.type(torch.long).to(device) for k,v in batch.items()}
        optimizer.zero_grad()
        outputs = model(**batch)
        loss, logits = outputs[:2]
        total_loss += loss.item()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()
        predictions_labels += logits.argmax(axis=-1).flatten().tolist()
    avg_epoch_loss = total_loss / len(dataloader)
    return predictions_labels, true_labels, avg_epoch_loss

# Function for validation 
def validate(dataloader, device):
    global model
    model.eval()
    predictions_labels = []
    true_labels = []
    total_loss = 0
    
    for batch in tqdm(dataloader, total=len(dataloader)):
        true_labels += batch['labels'].numpy().flatten().tolist()
        batch = {k:v.type(torch.long).to(device) for k,v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)
            loss, logits = outputs[:2]
            total_loss += loss.item()
            predictions_labels += logits.argmax(axis=-1).flatten().tolist()
    avg_epoch_loss = total_loss / len(dataloader)
    return predictions_labels, true_labels, avg_epoch_loss

def predict(dataloader, device):
    global model
    model.eval()
    predictions_labels = []
    
    for batch in tqdm(dataloader, total=len(dataloader)):
        batch = {k:v.type(torch.long).to(device) for k,v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)
            _, logits = outputs[:2]
            predictions_labels += logits.argmax(axis=-1).flatten().tolist()
    return predictions_labels   

print("done")

done


In [23]:
###code from Kaggle notebook
print('Loading gpt-2 model')
model_config = GPT2Config.from_pretrained(pretrained_model_name_or_path='gpt2', num_labels=2)

print('Loading tokenizer...')
tokenizer = GPT2Tokenizer.from_pretrained(pretrained_model_name_or_path='gpt2')
tokenizer.padding_side = "left"
tokenizer.pad_token = tokenizer.eos_token

print('Loading model...')
model = GPT2ForSequenceClassification.from_pretrained(pretrained_model_name_or_path='gpt2', config=model_config)
model.resize_token_embeddings(len(tokenizer)) 
model.config.pad_token_id = model.config.eos_token_id
model.to(device)

print("done")

Loading gpt-2 model


Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Loading tokenizer...


Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Loading model...


Downloading:   0%|          | 0.00/523M [00:00<?, ?B/s]

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


done


In [24]:
gpt2_collator = GPT2_collator(tokenizer=tokenizer, max_seq_len=max_length)

# Prepare training data
processed_data = myTrainData_df
train_data = DatasetCreator(processed_data, train=True)
train_dataloader = DataLoader(train_data, batch_size=batch_size, shuffle=True, collate_fn=gpt2_collator)

# Prepare validation data
val_processed = myTestData_df
val_data = DatasetCreator(val_processed, train=True)
val_dataloader = DataLoader(val_data, batch_size=batch_size, shuffle=True, collate_fn=gpt2_collator)

print('done')

done


In [25]:
optimizer = AdamW(model.parameters(), lr = 5e-5, eps = 1e-8, weight_decay=0.01)
print('test1')
total_steps = len(train_dataloader) * epochs
print('test2')
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps = 0, num_training_steps = total_steps)
print('test3')
loss = []
accuracy = []
val_loss_list = []
val_accuracy_list = []

for epoch in tqdm(range(epochs)):
    train_labels, true_labels, train_loss = train(train_dataloader, optimizer, scheduler, device)    
    print('test')
    train_acc = accuracy_score(true_labels, train_labels) 
    print('epoch: %.2f train accuracy %.2f' % (epoch, train_acc))
    loss.append(train_loss)
    accuracy.append(train_acc)

    val_labels, val_true_labels, val_loss = validate(val_dataloader, device)
    val_acc= accuracy_score(val_true_labels, val_labels)
    print('epoch: %.2f validation accuracy %.2f' % (epoch, val_acc))
    val_loss_list.append(val_loss)
    val_accuracy_list.append(val_acc)

test1
test2
test3


/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/782 [00:00<?, ?it/s]

test
epoch: 0.00 train accuracy 0.78


...

epoch: 2.00 validation accuracy 0.83


  0%|          | 0/782 [00:00<?, ?it/s]

test
epoch: 3.00 train accuracy 0.93


  0%|          | 0/782 [00:00<?, ?it/s]

epoch: 3.00 validation accuracy 0.83


In [27]:
import plotly.express as px
import plotly.graph_objects as go
# Print train and validation loss
fig_loss = go.Figure()

fig_loss.add_trace(go.Scatter(x=[*range(0,len(loss),1)], y=loss,
                              mode='lines',
                              name='train_loss'))
fig_loss.add_trace(go.Scatter(x=[*range(0,len(loss),1)], y=val_loss_list,
                              mode='lines',
                              name='validation loss'))

# Print train and validation accuracy
fig_acc = go.Figure()

fig_acc.add_trace(go.Scatter(x=[*range(0,len(accuracy),1)], y=accuracy,
                              mode='lines',
                              name='train accuracy'))
fig_acc.add_trace(go.Scatter(x=[*range(0,len(accuracy),1)], y=val_accuracy_list,
                              mode='lines',
                              name='validation accuracy'))

fig_loss.show()
fig_acc.show()